In [2]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import torch
from torch import nn
from matplotlib.pyplot import imread
from skimage.transform import resize
from skimage import img_as_ubyte
from PIL import Image
import torchvision.transforms as transforms



In [3]:
#model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
#model = torch.hub.load('pytorch/vision:v0.6.0', 'deeplabv3_resnet101', pretrained=True)
#yolov_model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True,force_reload=True).fuse().autoshape()  # for PIL/cv2/np inputs and NMS


In [4]:
torch.hub.list('zhanghang1989/ResNeSt', force_reload=True)
model = torch.hub.load('zhanghang1989/ResNeSt', 'resnest50', pretrained=True)
model.eval()

Downloading: "https://github.com/zhanghang1989/ResNeSt/archive/master.zip" to /root/.cache/torch/hub/master.zip
Using cache found in /root/.cache/torch/hub/zhanghang1989_ResNeSt_master


ResNet(
  (conv1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  )
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): SplAtConv2d(
        (conv): Conv2d

In [5]:
print(model)

ResNet(
  (conv1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  )
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): SplAtConv2d(
        (conv): Conv2d

In [6]:
IMG_PATH = "./kite.jpg"

In [7]:
device = torch.device("cpu")

In [8]:
def output(my_model, image_path):
    img = imread(image_path)
    img = img_as_ubyte(resize(img, (256,256)))
    temp = img
    img = img.transpose(2, 0, 1)
    img = img / 255
    img = torch.FloatTensor(img).to(device)
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                    std=[0.229, 0.224, 0.225])
    transform = transforms.Compose([normalize])
    image = transform(img)  # (3, 256, 256)
    # temp = image.transpose(1,2,3)
    # plt.imshow(temp)

    # Encode
    image = image.unsqueeze(0)  # (1, 3, 256, 256)
    res = my_model(image)  # (1, enc_image_size_h, enc_image_size_w, encoder_dim)
    print(res.shape)
    return res

In [9]:
layers = list(model.children())[:-2]
resnest_extractor = nn.Sequential(*layers) 


In [10]:
resnest_children_li= list(resnest_extractor.children() ) [4:]

In [11]:
print(len(resnest_children_li))

4


In [12]:
print(resnest_children_li)

[Sequential(
  (0): Bottleneck(
    (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): SplAtConv2d(
      (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=2, bias=False)
      (bn0): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (fc1): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
      (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (fc2): Conv2d(32, 128, kernel_size=(1, 1), stride=(1, 1))
      (rsoftmax): rSoftMax()
    )
    (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (downsample): Sequential(
      (0): AvgPool2d(kernel_size=1, stride=1, padding=0)
     

In [13]:
model_extractor.eval()

NameError: name 'model_extractor' is not defined

In [ ]:
print(output(model_extractor, IMG_PATH))

In [ ]:
"""
layers_1 = list(model.children())[:-1]
layers_2 = list(model.children())[-1].box_roi_pool
model1= nn.Sequential(
    *layers_1)
""" 

In [ ]:
in_features = model.roi_heads.box_predictor.cls_score.in_features

In [ ]:
print(in_features)

In [ ]:
class Tail(nn.Module):

    def __init__(self, in_channels, encoder_embedding_dim=196):
        super(Tail, self).__init__()
        self.linear1 = nn.Linear(in_channels, 8192)
        self.linear2 = nn.Linear(8192, encoder_embedding_dim)
        self.conv_layer= nn.Conv2d(1000,2048, 1)

    def forward(self, x):
        #if x.dim() == 4:
           #assert list(x.shape[2:]) == [1, 1]
        x = x.flatten(start_dim=1)
        x = self.linear1(x)
        x = self.linear2(x)
        
        #Here, res is 1000x4096
        
        x=torch.reshape(x,(1000,14,14))
        print(x.shape)
        
        x=torch.unsqueeze(x,0)
        
        #Of dimensions 2048x14x14
        res= self.conv_layer(x)

        print(res.shape)
        

        return res,res


In [ ]:
model.roi_heads.box_predictor = Tail(in_features, 196)


In [ ]:
print(model)

In [ ]:
print(output(model,IMG_PATH ))

In [ ]:
"""
class my_Faster_RCNN:
    def __init__():
        self.root= nn.S
        
    def forward(x):
        
        
    def backward():
        pass
        
"""

In [23]:
densenet= torch.hub.load('pytorch/vision:v0.6.0', 'densenet121', pretrained=True)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.6.0


In [24]:
densenet.eval()

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [25]:
layers = list(densenet.children())
print(len(layers))

2


In [26]:
layers = layers[:-1]

densenet_extractor = nn.Sequential(*layers,nn.Conv2d(1024, 2048, 1) ) 

In [27]:
print(output(densenet_extractor,IMG_PATH ) )

torch.Size([1, 2048, 8, 8])
tensor([[[[ 1.0375e-01,  1.2112e-02,  3.3136e-01,  ..., -2.5842e-02,
            1.7738e-01,  2.5774e-01],
          [ 5.3388e-02, -4.6131e-01, -3.2472e-01,  ...,  1.0247e-01,
            4.7731e-01,  5.9944e-01],
          [ 2.5193e-01, -4.9459e-02, -3.4655e-01,  ...,  1.5370e+00,
            1.1609e+00,  3.0951e-01],
          ...,
          [ 6.8394e-02,  2.2977e-01,  3.0588e-01,  ...,  1.2460e+00,
            5.5622e-01, -3.3569e-01],
          [ 1.3168e-01,  1.7645e-01,  3.4066e-01,  ...,  9.8505e-01,
            8.2884e-01,  8.2273e-01],
          [ 1.7859e-01,  1.9244e-01,  1.2069e-01,  ...,  9.6542e-01,
            1.3863e+00,  1.3278e+00]],

         [[-6.4733e-02,  5.7488e-02, -8.6996e-02,  ...,  4.8534e-01,
            6.2804e-01,  2.8794e-01],
          [ 1.5398e-02, -4.1887e-02, -2.5370e-01,  ...,  1.2471e-01,
            6.3196e-01,  5.4307e-01],
          [ 5.9576e-01,  2.2188e-01, -6.4323e-01,  ...,  1.8810e+00,
            2.2500e+00,  4.614

In [20]:
densenet_extractor

[Sequential(
   (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
   (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (relu0): ReLU(inplace=True)
   (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
   (denseblock1): _DenseBlock(
     (denselayer1): _DenseLayer(
       (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (relu1): ReLU(inplace=True)
       (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
       (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (relu2): ReLU(inplace=True)
       (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
     )
     (denselayer2): _DenseLayer(
       (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (relu1): ReLU(inplace=True)
       (conv1): 

In [28]:
densenet_extractor[1]

Conv2d(1024, 2048, kernel_size=(1, 1), stride=(1, 1))

In [65]:
from torchvision.models.mobilenet import mobilenet_v2

In [67]:
mobilenet = torchvision.models.mobilenet_v2(pretrained=True)
#layers = list(mobilenet.children())[:-1]

In [69]:
mobilenet_layers_list= list(mobilenet.children() ) 

In [70]:
print(len(mobilenet_layers_list))

2


In [71]:
len(mobilenet_layers_list[0])

19

In [72]:
print( mobilenet_layers_list[1] )

Sequential(
  (0): Dropout(p=0.2, inplace=False)
  (1): Linear(in_features=1280, out_features=1000, bias=True)
)


In [75]:
layers = list(mobilenet.children())[:-1]
mobilenet_extractor = nn.Sequential(*layers) 

In [76]:
print(mobilenet_extractor)

Sequential(
  (0): Sequential(
    (0): ConvBNReLU(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, trac

In [77]:
mobilenet_layers_list= list(mobilenet_extractor.children() ) 

In [78]:
print(len(mobilenet_layers_list))

1


In [82]:
wide_resnet = torch.hub.load('pytorch/vision:v0.6.0', 'wide_resnet50_2', pretrained=True)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.6.0


In [83]:
wide_resnet.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), strid

In [80]:
print(wide_resnet)

NameError: name 'wide_resnet' is not defined

In [84]:
wide_resnet_layers = list(wide_resnet.children())[:-1]
wide_resnet_extractor = nn.Sequential(*wide_resnet_layers) 

In [85]:
wide_resnet_layers_list= list(wide_resnet_extractor.children() ) 

In [86]:
print(len(wide_resnet_layers_list))

9


In [87]:
print(output(wide_resnet_extractor,IMG_PATH ))

torch.Size([1, 2048, 1, 1])
tensor([[[[0.1776]],

         [[0.9333]],

         [[0.1735]],

         ...,

         [[0.4940]],

         [[0.2115]],

         [[0.0381]]]], grad_fn=<ViewBackward>)


In [93]:
from torchvision.models.squeezenet import squeezenet1_0

In [100]:
squeezenet = squeezenet1_0(pretrained=True)

In [104]:
layers = list(squeezenet.children())[:-1]
print(len(layers))
squeezenet_extractor = nn.Sequential(*layers,nn.Conv2d(512, 2048, 1) )

1


In [105]:
print(squeezenet_extractor)

Sequential(
  (0): Sequential(
    (0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (3): Fire(
      (squeeze): Conv2d(96, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (4): Fire(
      (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (5): Fire(
      (squ

In [106]:
print(output(squeezenet_extractor,IMG_PATH ))

torch.Size([1, 2048, 15, 15])
tensor([[[[ 2.2087e-01, -6.8622e-01, -1.3021e+00,  ...,  1.7556e+00,
            5.5869e-01,  1.2005e+00],
          [-2.8603e+00, -4.6955e+00, -9.8684e+00,  ...,  2.6083e-01,
            4.2224e-01,  8.6833e-01],
          [-1.7464e-01, -2.3426e+00, -4.6581e+00,  ..., -2.8705e+00,
           -8.2767e-01,  3.1280e-01],
          ...,
          [-1.8974e+00,  2.3989e-01, -1.8786e+00,  ...,  1.0062e+01,
            9.2250e+00, -2.7699e-01],
          [-1.6212e+00, -3.4220e+00, -5.1781e+00,  ...,  1.1444e+01,
            7.7455e+00,  2.4342e+00],
          [ 2.4531e+00, -1.3200e+00, -3.0738e+00,  ...,  1.9935e+01,
            2.0467e+01,  1.0611e+01]],

         [[ 5.6039e+00,  3.9098e+00, -8.1085e-01,  ...,  3.4211e-01,
           -5.8473e-01, -4.3502e-01],
          [ 4.4957e+00,  3.9756e+00,  1.3815e+00,  ...,  2.8712e+00,
            9.8941e-01,  2.2164e+00],
          [ 1.0510e+00,  3.1131e+00,  2.8737e+00,  ..., -3.3843e+00,
            2.7866e-01,  9.9

In [109]:
children= list(squeezenet_extractor.children())

In [110]:
print(len(children))

2


In [111]:
children[0]

Sequential(
  (0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2))
  (1): ReLU(inplace=True)
  (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (3): Fire(
    (squeeze): Conv2d(96, 16, kernel_size=(1, 1), stride=(1, 1))
    (squeeze_activation): ReLU(inplace=True)
    (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
    (expand1x1_activation): ReLU(inplace=True)
    (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (expand3x3_activation): ReLU(inplace=True)
  )
  (4): Fire(
    (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
    (squeeze_activation): ReLU(inplace=True)
    (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
    (expand1x1_activation): ReLU(inplace=True)
    (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (expand3x3_activation): ReLU(inplace=True)
  )
  (5): Fire(
    (squeeze): Conv2d(128, 32, kernel_size=(1, 1), stride=(1, 1))
   

In [112]:
len(children[0])

13

In [116]:
for c in children[0]:
    for p in c.parameters():
        print(p)
        p.requires_grad = True
for p in children[1].parameters():
    p.requires_grad = True

Parameter containing:
tensor([[[[-7.8596e-03,  3.6351e-02,  5.9548e-02,  ..., -1.1042e-02,
           -3.1383e-02, -4.1490e-02],
          [-1.8588e-02,  4.4232e-02,  2.5275e-01,  ...,  1.1751e-01,
            7.1817e-02,  8.5236e-03],
          [-7.3618e-02, -1.4523e-01,  2.1080e-01,  ..., -5.9453e-02,
            7.0713e-02, -5.2098e-02],
          ...,
          [ 5.1393e-02, -1.1664e-01,  2.1728e-01,  ..., -5.8746e-02,
            2.0013e-01,  2.5179e-02],
          [-2.3340e-02, -1.1444e-01,  4.0975e-02,  ..., -1.8714e-01,
            3.3543e-02,  7.6814e-03],
          [ 5.9402e-04, -2.9829e-02,  9.1360e-02,  ..., -8.1534e-02,
            4.8496e-03, -5.4964e-03]],

         [[-4.6941e-03,  4.9783e-02,  6.0962e-02,  ...,  1.0213e-02,
            9.0631e-03, -1.8218e-02],
          [-3.8541e-02,  3.9837e-02,  2.4669e-01,  ...,  1.4264e-01,
            1.4346e-01,  4.6988e-02],
          [-1.0086e-01, -1.8138e-01,  1.8306e-01,  ..., -1.2363e-01,
            8.7764e-02, -5.1958e-02]

In [118]:
print(list(squeezenet_extractor.named_parameters()))

[('0.0.weight', Parameter containing:
tensor([[[[-7.8596e-03,  3.6351e-02,  5.9548e-02,  ..., -1.1042e-02,
           -3.1383e-02, -4.1490e-02],
          [-1.8588e-02,  4.4232e-02,  2.5275e-01,  ...,  1.1751e-01,
            7.1817e-02,  8.5236e-03],
          [-7.3618e-02, -1.4523e-01,  2.1080e-01,  ..., -5.9453e-02,
            7.0713e-02, -5.2098e-02],
          ...,
          [ 5.1393e-02, -1.1664e-01,  2.1728e-01,  ..., -5.8746e-02,
            2.0013e-01,  2.5179e-02],
          [-2.3340e-02, -1.1444e-01,  4.0975e-02,  ..., -1.8714e-01,
            3.3543e-02,  7.6814e-03],
          [ 5.9402e-04, -2.9829e-02,  9.1360e-02,  ..., -8.1534e-02,
            4.8496e-03, -5.4964e-03]],

         [[-4.6941e-03,  4.9783e-02,  6.0962e-02,  ...,  1.0213e-02,
            9.0631e-03, -1.8218e-02],
          [-3.8541e-02,  3.9837e-02,  2.4669e-01,  ...,  1.4264e-01,
            1.4346e-01,  4.6988e-02],
          [-1.0086e-01, -1.8138e-01,  1.8306e-01,  ..., -1.2363e-01,
            8.7764e-